In [18]:
import pandas as pd
import numpy as np
import re
from collections import Counter

from konlpy.tag import Okt
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_ka = pd.read_csv('./src/comments_kakao_300000.csv')
df_sam = pd.read_csv('./src/comments_samsung_200000.csv')

In [3]:
df = df_ka.append(df_sam)
df = df.reset_index(drop=True)
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (500000, 5)


,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-03,개미들 조련하는법,26,1,0
1,2022-05-03,6만원,38,1,0
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2
3,2022-05-03,안티 살발하다,34,2,0
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1


In [4]:
df = df[df['댓글'] != 'error']
df = df.dropna()
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (499984, 5)


,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-03,개미들 조련하는법,26,1,0
1,2022-05-03,6만원,38,1,0
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2
3,2022-05-03,안티 살발하다,34,2,0
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1


In [5]:
df['정제된 댓글'] = df['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ')
df['정제된 댓글'] = df['정제된 댓글'].str.replace('답글:',' ')
df['정제된 댓글'] = df['정제된 댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
df = df[df['정제된 댓글'] != '']
df = df.reset_index(drop=True)
df = df[['댓글','정제된 댓글']]
print('데이터 크기: ',df.shape)
df.head()

데이터 크기:  (493864, 2)


,댓글,정제된 댓글
0,개미들 조련하는법,개미들 조련하는법
1,6만원,만원
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상
3,안티 살발하다,안티 살발하다
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신


In [7]:
okt = Okt()
tag_list = ['Noun','Verb','Adjective','VerbPrefix']
tokenized_data = []

for i in range(df.shape[0]):
    tokenized_sentence = okt.pos(df['정제된 댓글'][i], stem=True) # 토큰화
    tag_checked_sentence = []
    for j in tokenized_sentence:
        x,y = j
        if y in tag_list:
            tag_checked_sentence.append(x)
    tokenized_data.append(tag_checked_sentence)
    print(f'\r{i+1}개 형태소분석 완료.',end='')
df['토큰화 댓글'] = tokenized_data
# df = df[df['토큰화 댓글'].str.len() > 1]
df = df.reset_index(drop=True)
df = df[df['토큰화 댓글'] != '[]']
print('\n데이터 크기: ',df.shape)
df.head()d

493864개 형태소분석 완료.
데이터 크기:  (493864, 3)


,댓글,정제된 댓글,토큰화 댓글
0,개미들 조련하는법,개미들 조련하는법,"[개미, 조련, 하다, 법]"
1,6만원,만원,[만원]
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"[카카오, 끝물, 이다, 사실]"
3,안티 살발하다,안티 살발하다,"[안티, 살발]"
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"[이렇다, 호로, 개미, 지옥탕, 만들다]"


In [43]:
df = df.drop_duplicates('토큰화 댓글')
df = df.reset_index(drop=True)
print('\n데이터 크기: ',df.shape)
df.head()


데이터 크기:  (362544, 3)


,댓글,정제된 댓글,토큰화 댓글
0,개미들 조련하는법,개미들 조련하는법,"['개미', '조련', '하다', '법']"
1,6만원,만원,['만원']
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']"
3,안티 살발하다,안티 살발하다,"['안티', '살발']"
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']"


In [44]:
# df.to_csv('./src/tokenized_comment.csv', index=False)

In [45]:
df = pd.read_csv('./src/tokenized_comment.csv')
print('\n데이터 크기: ',df.shape)
df.head()


데이터 크기:  (362544, 3)


,댓글,정제된 댓글,토큰화 댓글
0,개미들 조련하는법,개미들 조련하는법,"['개미', '조련', '하다', '법']"
1,6만원,만원,['만원']
2,카카오는 끝물입니다 사실상,카카오는 끝물입니다 사실상,"['카카오', '끝물', '이다', '사실']"
3,안티 살발하다,안티 살발하다,"['안티', '살발']"
4,이런 호로개미지옥탕을 만드신,이런 호로개미지옥탕을 만드신,"['이렇다', '호로', '개미', '지옥탕', '만들다']"


In [61]:
good = []
for i in range(df.shape[0]):
    comment = eval(df['토큰화 댓글'][i])
    if '좋다' in comment:
        good.append(comment)

In [65]:
bad = []
for i in range(df.shape[0]):
    comment = eval(df['토큰화 댓글'][i])
    if '망하다' in comment or '밉다' in comment:
        bad.append(comment)

In [66]:
len(good)

4099

In [67]:
len(bad)

3592